In [44]:
import cv2
import torchvision.transforms as transforms
import torch
import pandas as pd
from os import listdir
from os.path import isfile, join
import sys
import numpy as np
import math

In [21]:
def processImages(img_path,  size_target=256):
    img = cv2.imread(img_path)

    size_target = 450
    img = cv2.resize(img, (size_target, size_target), interpolation=cv2.INTER_LINEAR)

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    x = y = (size_target - 299) // 2

    h = w = 299
    img = bgr[y:y + h, x:x + w]

    arr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    t_img = torch.tensor(arr).unsqueeze(0).float()

    t_img /= 255.0


    return t_img

In [22]:
path_to_data = '/Users/janbraunsdorff/Studienarbeit-projekt/data'
path_to_validation_annotation = path_to_data + "/validation.csv"
path_to_training_annotation = path_to_data + "/training.csv"
path_to_validatoin_data = path_to_data + '/boneage-validation-dataset/'
path_to_training_data = path_to_data + '/boneage-training-dataset/'
path = path_to_data + '/pickel/v3'

In [38]:
count = 0
df_trainig = pd.read_csv(path_to_training_annotation)
for index, row in df_trainig.iterrows():
    img = processImages(path_to_training_data + str(row[0]) + ".png")
    count += torch.mean(img).item()
    if index % 200 == 0:
        print(index)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600


In [41]:
mean = count / float(len(df_trainig))

In [42]:
mean

0.2772987272627915

In [45]:
var = 0
for index, row in df_trainig.iterrows():
    img = processImages(path_to_training_data + str(row[0]) + ".png")
    var += math.pow(torch.mean(img).item() - mean, 2) *  0.003921568627

    if index % 200 == 0:
        print(index)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600


In [47]:
std = math.sqrt(var)

In [49]:
std, mean

(0.5228018736810739, 0.2772987272627915)

In [56]:
torch.Tensor([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])  * (std / 0.5) + (mean - 0.5) / 0.5

tensor([-0.4454, -0.3408, -0.2363, -0.1317, -0.0272,  0.0774,  0.1820,  0.2865,
         0.3911,  0.4956,  0.6002])